# [Module 4.1] Train XGBoost

---
## Train with XGBoost on SageMaker Cloud Instance
아 과정은 위의 전처리된 데이타를 가지고 실제 SageMaker Built-in Algorithm XGBoost를 이용하여 학습을 수행 합니다.<br>
실제의 학습 과정은 SageMaker Cloud Instance에서 실제 학습니 됩니다.


Built-in XGboost 알고리즘 컨테이너를 가져옵니다.

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [2]:
role = sagemaker.get_execution_role()

In [3]:
%store -r

Unable to restore variable 'scikit_learn_pre_process_model', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


In [4]:
%store

Stored variables and their in-db values:
bucket                                             -> 'sagemaker-us-east-2-057716757052'
inference_pipeline_model_name                      -> 'churn-inference-pipeline-2020-08-15-07-30-06'
pca_image_name                                     -> '057716757052.dkr.ecr.us-east-2.amazonaws.com/sage
pca_model_data                                     -> 's3://sagemaker-us-east-2-057716757052/Scikit-pca-
prefix                                             -> 'sagemaker/customer-churn'
preprocess_model                                   -> 'sagemaker-scikit-learn-2020-08-14-13-43-47-526'
preprocessed_pca_train_path                        -> 's3://sagemaker-us-east-2-057716757052/Scikit-pca-
preprocessed_pca_validation_path                   -> 's3://sagemaker-us-east-2-057716757052/Scikit-pca-
preprocessed_train_path                            -> 's3://sagemaker-us-east-2-057716757052/sagemaker/c
preprocessed_validation_path                       -> 's3://

In [5]:
import boto3

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


S3에 있는 Train, Validation 전처리된 (Features) 데이타의 경로 및 파일 포맷등을 지정하는 오브젝트를 생성 합니다.

In [6]:
s3_input_train_processed = sagemaker.session.s3_input(
    preprocessed_pca_train_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("S3 Train input: \n")
print(s3_input_train_processed.config)
s3_input_validation_processed = sagemaker.session.s3_input(
    preprocessed_pca_validation_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("\nS3 Validation input: \n")
print(s3_input_validation_processed.config)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


S3 Train input: 

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-15-07-44-44-619-2020-08-15-07-44-44-619', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}

S3 Validation input: 

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-validation-output/pca-2020-08-15-07-44-52-406-2020-08-15-07-44-52-406', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


#### 아래는 약 5분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [13])로 바뀔 때까지 기다려 주세요

In [ ]:
sess = sagemaker.Session()
instance_type = 'ml.m4.2xlarge'


xgb = sagemaker.estimator.Estimator(container, # Built-in XGBoost Container
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess
                                   )
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100,
                       )


xgb.fit({'train': s3_input_train_processed, 'validation': s3_input_validation_processed}) 

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


2020-08-15 07:45:39 Starting - Starting the training job.

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [ ]:
xgb_model_data = xgb.model_data
xgb_image_name = xgb.image_name
print("xgb_model_data: \n", xgb_model_data)
print("xgb_image_name: \n", xgb_image_name)

%store xgb_model_data
%store xgb_image_name